# Engineer
Here, we will engineer our features as follows:
* Convert numeric data to categorical
    * MoSold
    * MSSubClass
* Convert categorical data with orderings (likert scale type data) into ordinal data
E.g.,  `GarageQual`: NoGarage->0, Po->1, Fa->2, TA->3, Gd->4, Ex->5
* Create new features
    * Derived
    E.g., `NumFloors` in a property
    * Indicators (booleans)
    E.g., `IsPUD` whether a property is in a PUD
* Collapse number of categories for categorical features on a case-by-case basis
E.g., `OverallQual` has scale 1-10. 1-3 are simplified to "bad", 4-6 are "normal", 7-10 are "good"
* Dummify categorical features

In [93]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
import pandas as pd
import numpy as np
import data_dict
import features

In [95]:
df = pd.read_csv("../data/cleaned.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)

## Convert numerical data to categorical
`YrSold`, `MoSold`, and `MSSubClass` are categorical data but are encoded as integers in the dataset

In [96]:
# Assume date sold is first of the month
days = np.ones(df.shape[0])
date_sold = pd.to_datetime(dict(year=df.YrSold, month=df.MoSold, day=days))
df['DateSold'] = date_sold

In [97]:
# NOTE: We will treat MoSold and YrSold as categorical because there are only 12 (Jan->Dec) and 5 values (2006->2010).
# This might not be the case if we had many years of sales data in our dataset
df['YrSold'] = df['YrSold'].astype(str)
df['MoSold'] = df.MoSold.map(data_dict.convert_mosold)
df['MSSubClass'] = df['MSSubClass'].astype(str)

# Because we write out to a csv, we need to make sure that YrSold and MSSubClass are read back as categorical in other files.
# So we prepend a string to these columns
df['YrSold'] = df['YrSold'].apply(lambda x: "Yr_" + x)
df['MSSubClass'] = df['MSSubClass'].apply(lambda x: "Dwelling_" + x)

## Convert categorical to ordinal

In [98]:
# Overall features
# NO OP: These are already ordinalized

In [99]:
# Exterior features
df['ExterQual'] = df.ExterQual.map(data_dict.convert_exterqual)
df['ExterCond'] = df.ExterCond.map(data_dict.convert_extercond)

In [100]:
# Basement features
df['BsmtQual'] = df.BsmtQual.map(data_dict.convert_bsmtqual)
df['BsmtCond'] = df.BsmtCond.map(data_dict.convert_bsmtcond)
df['BsmtExposure'] = df.BsmtExposure.map(data_dict.convert_bsmtexposure)
df['BsmtFinType1'] = df.BsmtFinType1.map(data_dict.convert_bsmtfintype)
df['BsmtFinType2'] = df.BsmtFinType2.map(data_dict.convert_bsmtfintype)

In [101]:
# Home Interior features
df['Functional'] = df.Functional.map(data_dict.convert_functional)
df['FireplaceQu'] = df.FireplaceQu.map(data_dict.convert_fireplacequ)
df['HeatingQC'] = df.HeatingQC.map(data_dict.convert_heatingqc)
df['KitchenQual'] = df.KitchenQual.map(data_dict.convert_kitchenqual)

In [102]:
# Land features
df['LandSlope'] = df.LandSlope.map(data_dict.convert_landslope)
df['LotShape'] = df.LotShape.map(data_dict.convert_lotshape)

In [103]:
# Garage features
df['GarageCond'] = df.GarageCond.map(data_dict.convert_garagecond)
df['GarageQual'] = df.GarageQual.map(data_dict.convert_garagequal)

In [104]:
# Road features
df['Street'] = df.Street.map(data_dict.convert_street)
df['PavedDrive'] = df.PavedDrive.map(data_dict.convert_paveddrive)
df['Alley'] = df.Alley.map(data_dict.convert_alley)

In [105]:
# Other features
df['Utilities'] = df.Utilities.map(data_dict.convert_utilities)
df['PoolQC'] = df.PoolQC.map(data_dict.convert_poolqc)

## New Features
New data we can create from existing data. Does not mean we need to drop old data columns

In [106]:
# See whether a property is in a PUD from the dwelling type
df['IsPUD'] = df.MSSubClass.map(data_dict.get_pud_indicator)

In [107]:
# TODO LotShape isRegular

In [108]:
# Whether the property is near bad conditions (a busy road or a railroad)
df['IsNearNegativeCondition1'] = df.Condition1.map(data_dict.get_bad_condition_indicator)
df['IsNearNegativeCondition2'] = df.Condition2.map(data_dict.get_bad_condition_indicator)
df['IsNearNegativeCondition'] = df.apply(lambda x: x['IsNearNegativeCondition1'] | x['IsNearNegativeCondition2'], axis=1)
df['IsNearNegativeCondition'] = df['IsNearNegativeCondition'].replace({True: 1, False: 0})
df.drop(['IsNearNegativeCondition1', 'IsNearNegativeCondition2'], axis=1, inplace=True)

# Whether the property is near good conditions (a park, green-belt, etc)
df['IsNearPositiveCondition1'] = df.Condition1.map(data_dict.get_good_condition_indicator)
df['IsNearPositiveCondition2'] = df.Condition2.map(data_dict.get_good_condition_indicator)
df['IsNearPositiveCondition'] = df.apply(lambda x: x['IsNearPositiveCondition1'] | x['IsNearPositiveCondition2'], axis=1)
df['IsNearPositiveCondition'] = df['IsNearPositiveCondition'].replace({True: 1, False: 0})
df.drop(['IsNearPositiveCondition1', 'IsNearPositiveCondition2'], axis=1, inplace=True)

In [109]:
# Check good/bad counts
print(f"{df.IsNearNegativeCondition.sum()} properties near an artery or railroad")
print(f"{df.IsNearPositiveCondition.sum()} properties near park, green-belt, etc")

164 properties near an artery or railroad
56 properties near park, green-belt, etc


In [110]:
# Get number of floors for the property from the dwelling type
df['NumFloors'] = df.MSSubClass.map(data_dict.get_num_floors)

In [111]:
# Total house living area = GrLivArea + TotalFinBsmtSF
# GrLivArea (1stFlr + 2nd Flr + Unfinished)
# BsmtLivArea = (FinSF1 + FinSF2)
df['TotalFinBsmtSF'] = df.BsmtFinSF1 + df.BsmtFinSF2
# Combine all the outdoor SF features
df['TotalOutdoorSF'] = df.ScreenPorch + df.WoodDeckSF + df.OpenPorchSF + df.EnclosedPorch

In [112]:
# Pool - only care if one exists
df['HasPool'] = (df.PoolArea > 1).astype(int)


## Collapse/Combine Features
* Collapse: Use smaller scales for Likert scale-type categorical features (and drop the larger scale feature)
* Combine: Convert multiple features into a new feature (and drop the others)

Use smaller scales. Featured prefixed with `Collapse_`

In [113]:
df['Collapse_MSSubClass'] = df.MSSubClass.map(data_dict.collapse_mssubclass)

Combine multiple features. Features prefixed with `Combined_`

In [114]:
# Bathrooms
df['Combine_BathroomsBsmt'] = df['BsmtFullBath'] + df['BsmtHalfBath']*.5
df['Combine_BathroomsAbvGrd'] = df['FullBath'] + df['HalfBath']*0.5
df['AllBathrooms'] = df['Combine_BathroomsBsmt'] + df['Combine_BathroomsAbvGrd']

In [115]:
# Age
df['Age'] = 2010 - df.YearBuilt
df['AgeRemod'] = 2010 - df.YearRemodAdd
df['AgeGarage'] = 2010 - df.GarageYrBlt
# IsRenovated
df['IsRenovated'] = df.apply(lambda x: (x['YearRemodAdd'] > x['YearBuilt']), axis=1)
df['IsRenovated'] = df['IsRenovated'].replace({True: 1, False: 0})
# Check
df[['Age', 'AgeRemod', 'AgeGarage', 'IsRenovated', 'YearBuilt']]

,Age,AgeRemod,AgeGarage,IsRenovated,YearBuilt
0,71,60,71.0,1,1939
1,26,26,26.0,0,1984
2,80,3,80.0,1,1930
3,110,7,70.0,1,1900
4,9,9,9.0,0,2001
...,...,...,...,...,...
2572,94,60,94.0,1,1916
2573,55,55,55.0,0,1955
2574,61,60,61.0,1,1949
2575,10,10,10.0,0,2000


## Drop Other Features
Drop features that have been identified via EDA to be
* causes of multicolinearity
* have low explanatory power (insignificant; having low p-values)

In [116]:
# TODO drop features

## Add Log Target (SalePrice)
From EDA, we found that the distributions of features are skewed when plotted against `SalePrice`

In [117]:
df['LogSalePrice'] = np.log(df.SalePrice)

#### Housekeeping
Check that we organized every feature in `features.py`

In [118]:
# Test that we have all features labeled by type of data
features.check_features(df)

98 #columns == 99 #features+PID+targets


In [119]:
# Manually check that these are the same
features.check_features2(df)

,df.columns,features
0,1stFlrSF,1stFlrSF
1,2ndFlrSF,2ndFlrSF
2,3SsnPorch,3SsnPorch
3,Age,Age
4,AgeGarage,AgeGarage
...,...,...
94,WoodDeckSF,WoodDeckSF
95,YearBuilt,YearBuilt
96,YearRemodAdd,YearRemodAdd
97,YrSold,YrSold


### Add location coordinates
We are missing some property coordinates (~75), but add them where possible

In [120]:
locations = pd.read_csv("../data/housing_geolocation.csv", index_col=0)
locations = locations[['PID', 'latitude', 'longitude']]
locations.drop_duplicates(inplace=True)

In [121]:
df = pd.merge(left=df, right=locations, how='left')
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].fillna(np.NAN)

## Add school districts
NOTE: `districts.R` reads `engineered.csv` (which this notebook creates) and gets the school district for a property

In [122]:
districts = pd.read_csv("../data/districts.csv")

In [123]:
df = pd.merge(left=df, right=districts, how='left')
df['district'].fillna("Unknown", inplace=True)

Save outputs

In [124]:
df.to_csv("../data/engineered.csv")